# <center><h1> 🦜🔗 📝 MEMORY - запускаем чат-режим!</h1></center>

### Оглавление ноутбука
<img src='https://assets.cdn.prod.twilio.com/images/CLK0PzsrPZu0uIGxgMbZ8V7HtCJC4myfgeCpb5fOLxUKo.width-1000.png' align="right" width="600" height="650" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">🚀 Введение </a></li>
<li><a href="#c2">📝 Memory - сохранение истории диалога с LLM</a></li><ul type="square">
<li><a href="#c3">ConversationBufferMemory </a>
<li><a href="#c4">ConversationBufferWindowMemory и ConversationTokenBufferMemory </a>
<li><a href="#c5">ConversationSummaryMemory</a></ul>
    
</ul></font></p>

### В этом ноутбуке рассмотрим подробнее работу с `Memory`.

<div class="alert alert-info">
    
Память позволяет LLM запоминать предыдущие взаимодействия с пользователем. По умолчанию LLM является `stateless`, что означает независимую обработку каждого входящего запроса от других взаимодействий. На самом деле, когда мы вводим запросы в интерфейсе ChatGPT, под капотом работает память. <br>
В `LangChain` есть различные реализации памяти, посмотрим как они функционируют.<br>

 ###  <center>  `ConversationBufferMemory` - сохраняет всю историю диалога

In [1]:
#!pip install langchain openai tiktoken -q

In [ ]:
# Для работы в колабе загрузите наш скрипт для использования ChatGPT на сервере курса!
#!wget https://raw.githubusercontent.com/a-milenkin/LLM_practical_course/main/notebooks/utils.py

In [18]:
import os
from getpass import getpass
from langchain.chains import ConversationChain, LLMChain
from langchain.memory import ConversationBufferMemory

In [2]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain.chat_models import ChatOpenAI

# # os.environ['OPENAI_API_KEY'] = "Введите ваш OpenAI API ключ"
# os.environ['OPENAI_API_KEY'] = getpass(prompt='Введите ваш OpenAI API ключ')

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [11]:
# Если используете ключ из курса, запустите эту ячейку
from utils import ChatOpenAI

#course_api_key= "Введите ваш OpenAI API ключ"
course_api_key = getpass(prompt='Введите ваш OpenAI API ключ')

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

Введите ваш OpenAI API ключ ········


In [13]:
# создаём цепочку диалога, в параметры передаём языковую модель и память
memory = ConversationBufferMemory()
conversation_buf_mem = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [15]:
# Спросим Как дела? Видим, что пока контекст пустой (ключ history)

conversation_buf_mem("Привет, ChatGPT! Меня зовут Иван. Как дела?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Привет, ChatGPT! Меня зовут Иван. Как дела?
AI:

> Finished chain.


{'input': 'Привет, ChatGPT! Меня зовут Иван. Как дела?',
 'history': '',
 'response': 'Привет, Иван! Я рад снова поболтать с тобой. Я всего лишь программа, но у меня все отлично - всегда готов к интересным разговорам. Как я могу помочь тебе сегодня?'}

In [6]:
conversation_buf_mem("Сможешь помочь мне в написании кода на Python?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Привет, ChatGPT! Меня зовут Иван. Как дела?
AI: Привет, Иван! Я рад познакомиться с тобой. У меня все хорошо, спасибо! Как твои дела?
Human: Сможешь помочь мне в написании кода на Python?
AI:

> Finished chain.


{'input': 'Сможешь помочь мне в написании кода на Python?',
 'history': 'Human: Привет, ChatGPT! Меня зовут Иван. Как дела?\nAI: Привет, Иван! Я рад познакомиться с тобой. У меня все хорошо, спасибо! Как твои дела?',
 'response': 'Конечно, я постараюсь помочь! Что именно тебе нужно сделать?'}

In [7]:
conversation_buf_mem("Как вывести на экран фразу 'Hello, world!' ?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Привет, ChatGPT! Меня зовут Иван. Как дела?
AI: Привет, Иван! Я рад познакомиться с тобой. У меня все хорошо, спасибо! Как твои дела?
Human: Сможешь помочь мне в написании кода на Python?
AI: Конечно, я постараюсь помочь! Что именно тебе нужно сделать?
Human: Как вывести на экран фразу 'Hello, world!' ?
AI:

> Finished chain.


{'input': "Как вывести на экран фразу 'Hello, world!' ?",
 'history': 'Human: Привет, ChatGPT! Меня зовут Иван. Как дела?\nAI: Привет, Иван! Я рад познакомиться с тобой. У меня все хорошо, спасибо! Как твои дела?\nHuman: Сможешь помочь мне в написании кода на Python?\nAI: Конечно, я постараюсь помочь! Что именно тебе нужно сделать?',
 'response': "Чтобы вывести на экран фразу 'Hello, world!' на Python, ты можешь использовать функцию print(). Просто напиши print('Hello, world!') и запусти программу."}

<div class="alert alert-info">
    
#### Видно, что благодаря запоминанию контекста, модель знала, что нам нужен ответ именно c синтаксисом языка Python.

In [8]:
# Посмотреть содержимое истории диалога можно с помощью атрибута buffer
# При использовании буферной памяти, в history просто записываются все предыдущие сообщения.
print(conversation_buf_mem.memory.buffer)

Human: Привет, ChatGPT! Меня зовут Иван. Как дела?
AI: Привет, Иван! Я рад познакомиться с тобой. У меня все хорошо, спасибо! Как твои дела?
Human: Сможешь помочь мне в написании кода на Python?
AI: Конечно, я постараюсь помочь! Что именно тебе нужно сделать?
Human: Как вывести на экран фразу 'Hello, world!' ?
AI: Чтобы вывести на экран фразу 'Hello, world!' на Python, ты можешь использовать функцию print(). Просто напиши print('Hello, world!') и запусти программу.


In [9]:
# Или воспользоваться методом load_memory_variables, так же можно обращаться напрямую к объекту memory
memory.load_memory_variables({})

{'history': "Human: Привет, ChatGPT! Меня зовут Иван. Как дела?\nAI: Привет, Иван! Я рад познакомиться с тобой. У меня все хорошо, спасибо! Как твои дела?\nHuman: Сможешь помочь мне в написании кода на Python?\nAI: Конечно, я постараюсь помочь! Что именно тебе нужно сделать?\nHuman: Как вывести на экран фразу 'Hello, world!' ?\nAI: Чтобы вывести на экран фразу 'Hello, world!' на Python, ты можешь использовать функцию print(). Просто напиши print('Hello, world!') и запусти программу."}

**Так же контекст (историю) можно задавать вручную или подгружать из файла.**

In [10]:
# Создадим новый буфер
memory = ConversationBufferMemory()

# загрузим диалог в память
memory.save_context({"input": "Привет!"}, 
                    {"output": "Как дела?"})

print(memory.buffer)

Human: Привет!
AI: Как дела?


In [11]:
memory.load_memory_variables({})

{'history': 'Human: Привет!\nAI: Как дела?'}

In [12]:
# Добавим продолжение диалога
memory.save_context({"input": "Спасибо, хорошо!"}, 
                    {"output": "Здорово!"})

# Буфер дополнился
memory.load_memory_variables({})

{'history': 'Human: Привет!\nAI: Как дела?\nHuman: Спасибо, хорошо!\nAI: Здорово!'}

<div class="alert alert-info">
    
**Заглянем под капот:**
* Каждое новое сообщение подается в специальный промпт
* В промпт подаются 2 переменные `history` и `input`
* В параметре `template` описан шаблон промпта, в которые вставляются переменные.
* Остальное делает ChatGPT 🪄

In [13]:
# Посмотрим на этот промпт
conversation_buf_mem.prompt

PromptTemplate(input_variables=['history', 'input'], template='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.\n\nCurrent conversation:\n{history}\nHuman: {input}\nAI:')

In [14]:
print(conversation_buf_mem.prompt.template)

The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
{history}
Human: {input}
AI:


<div class="alert alert-info">
    
Этот тип памяти можно использовать в задачах с короткими диалогами и небольшим контекстом.<br>
**Недостатки `ConversationBufferMemory`:**
* Чем длиннее история, тем больше токенов придется подавать на вход
* В случае платных моделей, это будет накладно по финансам
* Контекст моделей тоже ограничен по количеству токенов (около 4K в случае ChatGPT)

### <center> `ConversationBufferWindowMemory` и `ConversationTokenBufferMemory` - сохраняют лимитированную часть контекста диалога

<div class="alert alert-info">

Похожие, по своей сути, 2 типа памяти:
* `ConversationBufferWindowMemory` - сохраняет в буфере только последние `k` диалогов.
* `ConversationTokenBufferMemory` - использует лимит токенов, который определяется количеством слов в сохраненных сообщениях.

In [19]:
from langchain.memory import ConversationBufferWindowMemory

memory = ConversationBufferWindowMemory(k=1)               

memory.save_context({"input": "Привет!"}, 
                    {"output": "Как дела?"})
memory.save_context({"input": "Спасибо, хорошо!"}, 
                    {"output": "Здорово!"})


memory.load_memory_variables({}) # Видим, что при k=1, в буфере остался только последний диалог

{'history': 'Human: Спасибо, хорошо!\nAI: Здорово!'}

In [20]:
memory = ConversationBufferWindowMemory(k=1)
conversation_bufwin_mem = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

In [21]:
conversation_bufwin_mem("Привет, ChatGPT! Меня зовут Иван. Как дела?")

{'input': 'Привет, ChatGPT! Меня зовут Иван. Как дела?',
 'history': '',
 'response': 'Привет, Иван! У меня все отлично, спасибо за спрос. Как могу помочь тебе сегодня?'}

In [18]:
# так же можно обращаться к цепочке через метод predict и в аутпуте получим только ответ модели
conversation_bufwin_mem.predict(input="Сможешь помочь мне в написании кода на Python?")

'Конечно, я помогу тебе с написанием кода на Python! Что именно ты хотел бы написать?'

In [19]:
# Видим, что при k=1 первый диалог с именем не сохранился и модель не может ответить
conversation_bufwin_mem.predict(input="Как меня зовут?")

'Извините, но я не знаю, как вас зовут. Я только могу помочь с вопросами и задачами, связанными с кодированием на Python.'

In [20]:
from langchain.memory import ConversationTokenBufferMemory

# Инициализируем новый token-буфер, подаем в него модель и лимит токенов
# Модель подаём потому что модели по-разному токенизируют текст
memory = ConversationTokenBufferMemory(llm=openai_llm, max_token_limit=50)

memory.save_context({"input": "Привет!"}, 
                    {"output": "Как дела?"})
memory.save_context({"input": "Спасибо, хорошо!"}, 
                    {"output": "Здорово!"})

memory.load_memory_variables({})

{'history': 'AI: Как дела?\nHuman: Спасибо, хорошо!\nAI: Здорово!'}

## <center> 🗜🗣 `ConversationSummaryMemory` - сохраняет "смысл"
Этот вид памяти суммаризует историю разговора перед передачей её в параметр `history`. Таким образом, можно сократить объем текста, обрабатываемого моделью и тратить меньше токенов.

In [22]:
from langchain.chains.conversation.memory import ConversationSummaryMemory

# Здесь тоже подаём модель, так как она необходима для суммаризации текста(не обязательно использовать ChatGPT)
memory = ConversationSummaryMemory(llm=llm)

conversation_sum_mem = ConversationChain(
    llm=llm,
    memory=memory
)

# посмотрим как поменялся шаблон для промпта
print(conversation_sum_mem.memory.prompt.template)

Progressively summarize the lines of conversation provided, adding onto the previous summary returning a new summary.

EXAMPLE
Current summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good.

New lines of conversation:
Human: Why do you think artificial intelligence is a force for good?
AI: Because artificial intelligence will help humans reach their full potential.

New summary:
The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
END OF EXAMPLE

Current summary:
{summary}

New lines of conversation:
{new_lines}

New summary:


<div class="alert alert-info">
    
Теперь у нас появляется этап предобработки истории. На вход LLM мы подаем предыдущую историю (в виде `summary`), добавляем к ней текущий вывод модели (`new_lines`) и получаем новое состояние `summary`.

In [24]:
conversation_sum_mem("Привет, ChatGPT! Меня зовут Иван. Как дела?")

{'input': 'Привет, ChatGPT! Меня зовут Иван. Как дела?',
 'history': '',
 'response': 'Привет, Иван! Я рад, что ты обратился ко мне. Я великолепно! Чем могу помочь тебе сегодня?'}

In [25]:
# Видим в history, суммаризованый предыдущий диалог уже на английском
conversation_sum_mem("Сможешь помочь мне в написании кода на Python?")

{'input': 'Сможешь помочь мне в написании кода на Python?',
 'history': 'The human greets the AI and introduces themselves as Ivan. The AI responds with a greeting and expresses its pleasure in being approached. The AI asks Ivan how it can assist him today.',
 'response': 'Конечно, могу помочь! Что именно ты пытаешься сделать?'}

In [26]:
conversation_sum_mem("Как меня зовут?")

{'input': 'Как меня зовут?',
 'history': 'The human greets the AI and introduces themselves as Ivan. The AI responds with a greeting and expresses its pleasure in being approached. The AI asks Ivan how it can assist him today. Ivan asks the AI if it can help him with writing code in Python. The AI responds that it can help and asks Ivan what he is trying to accomplish.',
 'response': 'Простите, но я не могу знать ваше имя, так как мы общаемся только через текстовый интерфейс. Меня зовут OpenAI, и я здесь, чтобы помочь вам с кодированием на Python. В чем я могу вам помочь сегодня?'}

In [27]:
conversation_sum_mem("What is my name in English?")

{'input': 'What is my name in English?',
 'history': 'The human greets the AI and introduces themselves as Ivan. The AI responds with a greeting and expresses its pleasure in being approached. The AI asks Ivan how it can assist him today. Ivan asks the AI if it can help him with writing code in Python. The AI responds that it can help and asks Ivan what he is trying to accomplish. The human then asks the AI, "What is my name?" The AI apologizes and explains that it cannot know his name as they are only communicating through a text interface. The AI then introduces itself as OpenAI and offers to help with Python coding. It asks Ivan how it can assist him today.',
 'response': "I'm sorry, but I don't have access to your personal information, including your name. Therefore, I cannot determine what your name is in English. Can I help you with anything else?"}

<div class="alert alert-info">
    
### Другие виды памяти:

* `ConversationSummaryBufferMemory` - позволяет задать окно (в токенах), в рамках которого мы сохраняем диалоги в неизменной форме, а при превышении - суммаризируем.
* `ConversationKGMemory` - использует граф знаний для сохранения памяти.
* `ConversationEntityMemory` - сохраняет в памяти знания об определенном объекте.

# <center><h1>🔗 Chains 🔗 - продолжение следует...</h1></center>